In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import linalg as la
import pandas as pd

1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normal distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


In [19]:
from numpy import sqrt

N = np.random.randint(10,11)

mean_x1, std_x1 = 0 , sqrt(1)
x1 = np.random.randn(N,1)*std_x1+mean_x1 # mean 0 var 1

mean_x2, std_x2 = mean_x1+0 , sqrt(std_x1**2 + 3**2)
x2 = np.random.randn(N,1)*std_x2+mean_x2 # mean 0 var 10

mean_x3, std_x3 = 2*mean_x1+mean_x2 , sqrt(2**2*std_x1**2 + std_x2**2)
x3 = np.random.randn(N,1)*std_x3+mean_x3 # mean 0 var 14

df = pd.DataFrame(data=np.concatenate([x1,x2,x3], axis=1), columns=['x_1', 'x_2', 'x_3'])

# COV MATRIX OF DATASET
l, V = np.linalg.eig(np.cov(df))

print("normal: ", l)

# SVD
U, D, Vt = la.svd(df)

l_svd = D**2/(N-1)
V_svd = U

print("SVD: ", l_svd)

df

normal:  [ 7.10542736e-15+0.00000000e+00j  4.47082917e+01+0.00000000e+00j
  2.18359899e+01+0.00000000e+00j -7.93579349e-16+0.00000000e+00j
  1.38823468e-15+0.00000000e+00j  8.94743306e-16+0.00000000e+00j
  6.80199127e-16+0.00000000e+00j  8.98034896e-17+2.34200199e-16j
  8.98034896e-17-2.34200199e-16j -1.17182016e-17+0.00000000e+00j]
SVD:  [19.95695209  8.62648037  1.06644201]


,x_1,x_2,x_3
0,0.857213,-1.632160,-3.047619
1,0.221219,-4.848875,-4.056908
2,1.159569,4.852455,3.723159
3,1.617132,6.860943,-2.058051
4,-1.093078,-1.926173,-4.611888
5,-1.175832,1.186427,4.737917
6,1.241982,-0.523696,-3.223478
7,-1.305508,-1.502615,-1.038021
8,-1.292021,0.636911,-6.127066
9,-0.416482,-1.570363,-3.248359


2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [3]:
# get the dataset and its description on the proper data directory
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 